In [ ]:
import sage.combinat.finite_state_machine
sage.combinat.finite_state_machine.FSMOldProcessOutput = False
sage.combinat.finite_state_machine.FSMOldCodeTransducerCartesianProduct = False

# Automata and Transducers in SageMath

## Digit Expansions

* **decimal system**: base 10, digits 0, 1, ..., 9
* **binary system**: base 2, digits 0, 1
* base 2, digits -1, 0, 1 --> *redundancy*

### Non-adjacent Form (NAF)

* no consequtive non-zero digits in expansion
* examples
  * 3 = 1 + 2 = (1 1)<sub>2</sub> (standard binary) ... not a NAF
  * 3 = -1 + 4 = (-1 0 1)<sub>2</sub> ... a NAF

## Creating a Transducer from Scatch

In [ ]:
NAF1 = Transducer([('I', 0, 0, None), ('I', 1, 1, None),
                   (0, 0, 0, 0), (0, 1, 1, 0),
                   (1, 0, 0, 1), (1, 2, 1, -1),
                   (2, 1, 0, 0), (2, 2, 1, 0)], 
                  initial_states=['I'], final_states=[0], 
                  input_alphabet=[0, 1])
NAF = NAF1
NAF

In [ ]:
view(NAF)

In [ ]:
14.digits(base=2)

In [ ]:
NAF(14.digits(base=2))

In [ ]:
NAF.process(14.digits(base=2))

In [ ]:
NAF(14.digits(base=2) + [0, 0, 0])

In [ ]:
NAF = NAF.with_final_word_out(0)

In [ ]:
NAF(14.digits(base=2))

## Calculating the Non-adjacent Form with Less Thinking

In [ ]:
def NAF_transition(state_from, read):
    if state_from == 'I':
        write = None
        state_to = read
        return (state_to, write)
    current = 2*read + state_from
    if current % 2 == 0:
        write = 0
    elif current % 4 == 1:
        write = 1
    else:
        write = -1
    state_to = (current - write) / 2
    return (state_to, write)

NAF2 = Transducer(NAF_transition,
                  initial_states=['I'],
                  final_states=[0],
                  input_alphabet=[0, 1]).with_final_word_out(0)

NAF == NAF2

In [ ]:
NAF2(14.digits(base=2))

## A 3rd Construction of the Same Transducer

* (NAF of 2n) = (binary of 3n) &ndash; (binary of n)

In [ ]:
def f(state_from, read):
    current = 3*read + state_from
    write = current % 2
    state_to = (current - write) / 2
    return (state_to, write)

Triple = Transducer(f, input_alphabet=[0, 1],
                    initial_states=[0],
                    final_states=[0]).with_final_word_out(0)
Triple

In [ ]:
Triple(4.digits(base=2))

In [ ]:
Id = Transducer([(0, 0, 0, 0), (0, 0, 1, 1)],
                initial_states=[0], final_states=[0],
                input_alphabet=[0, 1])

In [ ]:
prebuiltId = transducers.Identity([0, 1])

In [ ]:
Combined_3n_n = Triple.cartesian_product(Id).relabeled()
Combined_3n_n

In [ ]:
Combined_3n_n(4.digits(base=2))

In [ ]:
def g(read0, read1):
    return ZZ(read0) - ZZ(read1)

Minus = transducers.operator(g, input_alphabet=[None, -1, 0, 1])

In [ ]:
prebuiltMinus = transducers.sub([-1, 0, 1])

In [ ]:
NAF3 = Minus(Combined_3n_n).relabeled()  # compositions of transducers

In [ ]:
NAF3(14.digits(base=2))

## An Automaton detecting NAFs

In [ ]:
view(NAF)

In [ ]:
A = NAF.output_projection()
A

In [ ]:
A([0])

In [ ]:
A([0, -1, 1])

In [ ]:
A([1, 0, 1])

In [ ]:
view(A)

In [ ]:
A = A.split_transitions()
A

In [ ]:
A.is_deterministic()

In [ ]:
A.determine_alphabets()
A = A.minimization().relabeled()
A

In [ ]:
A.is_deterministic()

## Combining Small Transducers to a Larger One: The 3/2-1/2-NAF



In [ ]:
NAF = NAF3
NAF3n = NAF(Triple)  # composition
Combined_NAF_3n_n = NAF3n.cartesian_product(NAF).relabeled()
T = Minus(Combined_NAF_3n_n).relabeled()  # composition
T

In [ ]:
T(14.digits(base=2))

In [ ]:
def value(digits):
    return sum(d * 2^(e-2) for e, d in enumerate(digits))
value(T(14.digits(base=2)))

## Again an Alternative Construction

In [ ]:
def minus(trans1, trans2):
    if trans1.word_in == trans2.word_in:
        return (trans1.word_in, 
                trans1.word_out[0] - trans2.word_out[0])
    else:
        raise LookupError

from itertools import izip_longest
def final_minus(state1, state2):
    return [x - y for x, y in 
        izip_longest(state1.final_word_out, state2.final_word_out, fillvalue=0)]

Talternative = NAF3n.product_FiniteStateMachine(
                           NAF, minus,
                           final_function=final_minus).relabeled()
Talternative == T

## Getting a Picture

In [ ]:
sage.combinat.finite_state_machine.setup_latex_preamble()
latex.mathjax_avoid_list('tikzpicture')
T.set_coordinates({
    0: (-2, 0.75),
    1: (0, -1),
    2: (-6, -1),
    3: (6, -1),
    4: (-4, 2.5),
    5: (-6, 5),
    6: (6, 5),
    7: (4, 2.5),
    8: (2, 0.75)})
T.latex_options(format_letter=T.format_letter_negative,
                accepting_where={
                  0: 'right',
                  1: 'below',
                  2: 'below',
                  3: 'below',
                  4: 60,
                  5: 'above',
                  6: 'above',
                  7: 120,
                  8: 'left'},
                accepting_show_empty=True)

view(latex(T))
latex(T)

## Weights

In [ ]:
def weight(state_from, read):
    write = ZZ(read != 0)
    return (0, write)

Weight = Transducer(weight, input_alphabet=srange(-2, 2+1),
                    initial_states=[0], final_states=[0])
Weight.add_transition((0, 0, None, None))
Weight

In [ ]:
prebuiltWeight = transducers.weight(srange(-2, 2+1))

In [ ]:
NAF = NAF2  # reset since modified above
W_binary = Weight(Id)
W_NAF = Weight(NAF)
W_T = Weight(T)

In [ ]:
expansion = 14.digits(base=2)
print "binary" , Id(expansion), W_binary(expansion), sum(W_binary(expansion))
print "NAF", NAF(expansion), W_NAF(expansion), sum(W_NAF(expansion))
print "T", T(expansion), W_T(expansion), sum(W_T(expansion))

## Also Possible: Adjacency Matrices

In [ ]:
var('y')
def am_entry(trans):
    return y^add(trans.word_out) / 2
W_T.adjacency_matrix(entry=am_entry)

## Asymptotic Analysis

In [ ]:
var('k')
W_binary.asymptotic_moments(k)

In [ ]:
W_NAF.asymptotic_moments(k)

In [ ]:
W_T.asymptotic_moments(k)